In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from itertools import combinations

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
from tqdm import tqdm

In [ ]:
dataset = load_dataset("BeIR/scifact", "corpus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

scifact.py:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.58M [00:00<?, ?B/s]

Generating corpus split:   0%|          | 0/5183 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 5183
    })
})

In [ ]:
dataset_q = load_dataset("BeIR/scifact", "queries")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

scifact.py:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

queries/queries/0000.parquet:   0%|          | 0.00/67.5k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/1109 [00:00<?, ? examples/s]

In [ ]:
dataset_q

DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 1109
    })
})

In [ ]:
dataset_qrel = load_dataset("BeIR/scifact-qrels", split="test")

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/919 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/339 [00:00<?, ? examples/s]

In [ ]:
dataset_qrel

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 339
})

In [ ]:
dataset_qrel[0]

{'query-id': 1, 'corpus-id': 31715818, 'score': 1}

In [ ]:
from collections import defaultdict

# Initialize a defaultdict to store query-doc pairs
qrels = defaultdict(dict)

# Iterate over the dataset and populate the dictionary
for qrel in dataset_qrel:
    qrels[qrel["query-id"]][qrel["corpus-id"]] = qrel["score"]

# Convert defaultdict back to a regular dictionary if needed
qrels = dict(qrels)

In [ ]:
qrels

{1: {31715818: 1},
 3: {14717500: 1},
 5: {13734012: 1},
 13: {1606628: 1},
 36: {5152028: 1, 11705328: 1},
 42: {18174210: 1},
 48: {13734012: 1},
 49: {5953485: 1},
 50: {12580014: 1},
 51: {45638119: 1},
 53: {45638119: 1},
 54: {49556906: 1},
 56: {4709641: 1},
 57: {4709641: 1},
 70: {5956380: 1, 4414547: 1},
 72: {6076903: 1},
 75: {4387784: 1},
 94: {1215116: 1},
 99: {18810195: 1},
 100: {4381486: 1},
 113: {6157837: 1},
 115: {33872649: 1},
 118: {6372244: 1},
 124: {4883040: 1},
 127: {21598000: 1},
 128: {8290953: 1},
 129: {27768226: 1},
 130: {27768226: 1},
 132: {7975937: 1},
 133: {38485364: 1, 6969753: 1, 17934082: 1, 16280642: 1, 12640810: 1},
 137: {26016929: 1},
 141: {6955746: 1, 14437255: 1},
 142: {10582939: 1},
 143: {10582939: 1},
 146: {10582939: 1},
 148: {1084345: 1},
 163: {18872233: 1},
 171: {12670680: 1},
 179: {16322674: 1, 27123743: 1, 23557241: 1, 17450673: 1},
 180: {16966326: 1},
 183: {12827098: 1},
 185: {18340282: 1},
 198: {2177022: 1},
 208: {13

In [ ]:
queries = {query["_id"]: query["text"] for query in dataset_q["queries"]}

In [ ]:
corpus = {doc["_id"]: doc['text'] for doc in dataset["corpus"]}

In [ ]:
print(f"Corpus size: {len(corpus)}")
print(f"Number of queries: {len(queries)}")
print(f"Number of query-document pairs: {len(qrels)}")

Corpus size: 5183
Number of queries: 1109
Number of query-document pairs: 300


In [ ]:
len(corpus)

5183

In [ ]:
# documents = list(corpus.values())


In [ ]:
# doc_ids = list(corpus.keys())

In [ ]:
# Load generative model (GPT-3.5 or FLAN-T5)
def load_generative_model(model_name="google/flan-t5-large"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

In [ ]:
# Segment document into chunks
def segment_document(document, chunk_size, overlap):
    tokens = document.split()
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    return chunks

In [ ]:
def generate_pseudo_queries(chunks, tokenizer, model, top_k_ques=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the same device (GPU or CPU)

    # Prepare the prompts
    prompts = [
        f"Generate a detailed and nuanced question focusing on the most significant aspects of the passage:\n\n{chunk}\n\n"
        for chunk in chunks
    ]

    # Tokenize the input
    inputs = tokenizer(prompts, truncation=True, padding=True, return_tensors="pt", max_length=512)
    # print(inputs)


    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate pseudo queries without gradients
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=20,
            num_return_sequences=top_k_ques,  # Number of queries to generate per chunk
            do_sample=True,  # Use sampling to generate diverse queries
            top_k=5         # Limit to the top 10 most probable next tokens
        )

    # Decode the generated sequences into queries
    queries = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Since `batch_decode` will return a flat list, we need to group them into batches of `top_k_ques`
    queries = [
        queries[i:i + top_k_ques]
        for i in range(0, len(queries), top_k_ques)
    ]

    return queries


In [ ]:

# Diversity filtering using semantic similarity
def filter_diverse_queries(queries, similarity_model, threshold=0.8):
    filtered_queries = []
    embeddings = similarity_model.encode(queries, convert_to_tensor=True)
    for i, query in enumerate(queries):
        is_redundant = any(
            util.cos_sim(embeddings[i], embeddings[j]) > threshold for j in range(len(filtered_queries))
        )
        if not is_redundant:
            filtered_queries.append(query)
    return filtered_queries


In [ ]:
def calculate_similarity_score(user_pseudo_query, document_pseudo_queries, similarity_model):
    """
    Calculate the similarity score for a query-document pair.

    Parameters:
    - query: The user query (string).
    - document_pseudo_queries: List of pseudo-queries for the document.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - max_similarity: Maximum similarity score between query and pseudo-queries.
    """
    # Compute embeddings for the query and pseudo-queries
    # query_embedding = similarity_model.encode(user_pseudo_query, convert_to_tensor=True)
    # # pseudo_query_embeddings = similarity_model.encode(document_pseudo_queries, convert_to_tensor=True)

    # # Compute cosine similarity scores
    # similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    similarity_model_gpu = similarity_model.to(device)

    # Compute embedding for the query on the specified device
    query_embedding = similarity_model_gpu.encode(
        user_pseudo_query,
        convert_to_tensor=True,
        device=device
    )

    if query_embedding.dim() == 1:
        query_embedding = query_embedding.unsqueeze(0)

    # Ensure pseudo-query embeddings are also on the same device
    document_pseudo_queries = document_pseudo_queries.to(device)
    # print(query_embedding.shape)
    # print(document_pseudo_queries.shape)

    # Compute cosine similarity scores
    similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)

    # Find the maximum similarity score
    max_similarity = torch.max(similarity_scores).item()
    return max_similarity



In [ ]:
def rank_documents_by_query(query, document_pseudo_queries_embeddings, similarity_model):
    """
    Rank documents based on the similarity of their pseudo-queries to the query.

    Parameters:
    - query: The user query (string).
    - document_queries_dict: Dictionary where keys are document IDs and values are lists of pseudo-queries.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - ranked_scores: List of tuples (document_id, score) sorted by descending scores.
    """
    document_scores = {}
    for doc_id, pseudo_queries_embeddings in document_pseudo_queries_embeddings.items():
        # print(doc_id)
        # print(pseudo_queries)
        # print("yoo")
        score = calculate_similarity_score(query, pseudo_queries_embeddings, similarity_model)
        document_scores[doc_id] = score

    # Sort documents by scores in descending order
    ranked_scores = sorted(document_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_scores

In [ ]:
# Model names
generative_model_name = "google/flan-t5-large"
similarity_model_name = "all-mpnet-base-v2"
chunk_size=500
overlap=0
threshold=0.8

In [ ]:
similarity_model = SentenceTransformer(similarity_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Main function to process the corpus
def process_corpus(corpus, generative_model_name, similarity_model, chunk_size, overlap, threshold):
    tokenizer, generative_model = load_generative_model(generative_model_name)

  # Initialize dictionaries to store the chunks and queries
    document_chunks = {}
    document_generated_queries = {}

    # Loop over each document in the corpus with tqdm progress bar
    for doc_id, document in tqdm(corpus.items(), desc="Processing Documents"):
        # Segment the document into chunks
        chunks = segment_document(document, chunk_size, overlap)

        # Store the chunks in a dictionary with the doc_id as the key
        document_chunks[doc_id] = chunks

        chunk_queries = []
        generated_queries = generate_pseudo_queries(chunks, tokenizer, generative_model)
        # print("Generated Queries")
        # print(generated_queries)
        # print("Generated Queries [0] ")
        # print(generated_queries[0])
        # break

        diverse_queries = filter_diverse_queries(generated_queries[0], similarity_model, threshold)
        chunk_queries.extend(diverse_queries)

        document_generated_queries[doc_id] = chunk_queries
    return document_generated_queries


In [ ]:
results = process_corpus(corpus, generative_model_name, similarity_model,chunk_size, overlap, threshold)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Processing Documents: 100%|██████████| 5183/5183 [1:23:18<00:00,  1.04it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(results)

5183

# NOTE :

SImilarly, you can use the above code to generate the psuedo query embeddings and save it per dataset.

These pseudo queries can be used for training the autoregressive model

# NOTE
Below is the code to call functions for generating the psuedo queries
You need not create the psuedo queries. Use the following link to find the generated psuedo queries which you can use to train the autoregressive model.

Link: https://drive.google.com/drive/folders/191D9QMsCVku2V1aCE0ZlkWvDqCzXlWQ3?usp=sharing

Check the files for their suffix to know which dataset they contain.

In [ ]:
import json

In [ ]:
# file_path ="document_generated_queries_withdiversequeries_flan-t5-large_scifact_DONOTRERUN.json"

# with open(file_path, "r") as file:
#   data = json.load(file)


In [ ]:
# with open("document_generated_queries_withdiversequeries_flan-t5-large_scifact.json", "w") as file:
#     json.dump(data, file, indent=4)
# print("Data saved as generated_queries.json")

In [ ]:
file_path = "/content/drive/MyDrive/646Project/sharedfiles/document_generated_queries_withdiversequeries_flan-t5-large_scifact.json"

with open(file_path, "r") as file:
  data = json.load(file)


In [ ]:
document_pseudo_queries = data

In [ ]:
queries_dict = list(qrels.items())[:100]

In [ ]:
len(queries_dict)

100

In [ ]:
# queries_dict = list(qrels.items())[:2]
test_dict = {}
document_pseudo_queries_embeddings={}
device = "cuda" if torch.cuda.is_available() else "cpu"
similarity_model_gpu = similarity_model.to(device)

for doc_id, pseudo_queries in tqdm(document_pseudo_queries.items()):
  # print(pseudo_queries)
  # print(len(pseudo_queries))
  pseudo_query_embeddings = similarity_model_gpu.encode(pseudo_queries, convert_to_tensor=True, device = device)
  document_pseudo_queries_embeddings[doc_id] = pseudo_query_embeddings
  # # print(doc_id)
  # print(pseudo_query_embeddings[0])
  # print(len(pseudo_query_embeddings))
  # # print(document_pseudo_queries_embeddings)
  # break



100%|██████████| 5183/5183 [02:01<00:00, 42.53it/s]


In [ ]:
queries

{'0': '0-dimensional biomaterials lack inductive properties.',
 '2': '1 in 5 million in UK have abnormal PrP positivity.',
 '4': '1-1% of colorectal cancer patients are diagnosed with regional or distant metastases.',
 '6': '10% of sudden infant death syndrome (SIDS) deaths happen in newborns aged less than 6 months.',
 '9': '32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.',
 '10': '4-PBA treatment decreases endoplasmic reticulum stress in response to general endoplasmic reticulum stress markers.',
 '11': '4-PBA treatment raises endoplasmic reticulum stress in response to general endoplasmic reticulum stress markers.',
 '12': '40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.',
 '14': "5'-nucleotidase metabolizes 6MP.",
 '15': '50% of patients exposed to radiation have activated markers of mesenchymal stem cells.',
 '17': '53% of perinatal mortality is due to low 

In [ ]:
qrels

{1: {31715818: 1},
 3: {14717500: 1},
 5: {13734012: 1},
 13: {1606628: 1},
 36: {5152028: 1, 11705328: 1},
 42: {18174210: 1},
 48: {13734012: 1},
 49: {5953485: 1},
 50: {12580014: 1},
 51: {45638119: 1},
 53: {45638119: 1},
 54: {49556906: 1},
 56: {4709641: 1},
 57: {4709641: 1},
 70: {5956380: 1, 4414547: 1},
 72: {6076903: 1},
 75: {4387784: 1},
 94: {1215116: 1},
 99: {18810195: 1},
 100: {4381486: 1},
 113: {6157837: 1},
 115: {33872649: 1},
 118: {6372244: 1},
 124: {4883040: 1},
 127: {21598000: 1},
 128: {8290953: 1},
 129: {27768226: 1},
 130: {27768226: 1},
 132: {7975937: 1},
 133: {38485364: 1, 6969753: 1, 17934082: 1, 16280642: 1, 12640810: 1},
 137: {26016929: 1},
 141: {6955746: 1, 14437255: 1},
 142: {10582939: 1},
 143: {10582939: 1},
 146: {10582939: 1},
 148: {1084345: 1},
 163: {18872233: 1},
 171: {12670680: 1},
 179: {16322674: 1, 27123743: 1, 23557241: 1, 17450673: 1},
 180: {16966326: 1},
 183: {12827098: 1},
 185: {18340282: 1},
 198: {2177022: 1},
 208: {13

In [ ]:
from openai import OpenAI
from nltk.corpus import wordnet
import nltk

# Download WordNet data if not already available
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Function to get synonyms for a word
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    return list(synonyms)

# Function to rephrase using GPT-4
# def rephrase_query_with_gpt(query, api_key):
#     prompt=f"Rephrase the following query to make it clearer and more comprehensive: {query}",
#     # openai.api_key = api_key
#     client = OpenAI(api_key=api_key)
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": prompt}],
#         max_tokens=50,
#         temperature=0.7
#     )
#     # response = openai.Completion.create(
#     #     engine="gpt-4o-mini",  # Use the appropriate GPT model engine
#     #     prompt=f"Rephrase the following query to make it clearer and more comprehensive: {query}",
#     #     max_tokens=50,
#     #     temperature=0.7
#     # )
#     # response.choices[0].message.content.strip("[]")
#     # return response.choices[0].text.strip("[]")
#     return response.json()

def rephrase_query_with_gpt(query, api_key):
    from openai import OpenAI

    # Ensure prompt is a string, not a tuple
    prompt = f"Rephrase the following query to make it clearer and more comprehensive: {query}"

    # Initialize the OpenAI client
    client = OpenAI(api_key=api_key)

    # Make the request to the Chat Completions endpoint
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Use the appropriate model
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0.7
    )

    # Extract and return the response text
    # return response.choices[0].message['content'].strip()
    # return response.json()
    # return response["choices"][0]["message"]["content"]
    return response.choices[0].message.content.strip()

# Main function to generate a pseudo query
def generate_pseudo_query(query, api_key):
    words = query.split()
    expanded_query = []

    # Add synonyms for each word
    for word in words:
        expanded_query.append(word)  # Include the original word
        synonyms = get_synonyms(word)
        if synonyms:
            expanded_query.extend(synonyms[:2])  # Add up to 2 synonyms for each word to avoid over-expansion

    # Create a combined query string
    expanded_query_str = " ".join(set(expanded_query))  # Remove duplicates by using `set`
    # print("Expanded Query: ",expanded_query_str)
    # Rephrase using GPT
    rephrased_query = rephrase_query_with_gpt(expanded_query_str, api_key)

    return rephrased_query

In [ ]:
api_key="API_KEY"

PSEUDO USER GENERATED QUERY REPLACE IN CELL BELOW. CHECK THE COMMENTED CODE LINE:
*user_pseudo_query = generate_pseudo_query(queries[query_id])*

In [ ]:
qrel_pseudo ={}
for query_id, doc_score in tqdm(queries_dict):

  user_pseudo_query = generate_pseudo_query(queries[str(query_id)],api_key)
  qrel_pseudo[str(query_id)]=user_pseudo_query
  # user_pseudo_query = queries[str(query_id)]
  doc_rank_for_query = rank_documents_by_query(user_pseudo_query, document_pseudo_queries_embeddings, similarity_model)
  # print(type(doc_rank_for_query))
  test_dict[query_id]={}
  for doc_id, score in doc_rank_for_query:
    test_dict[query_id][doc_id] = float(score)


100%|██████████| 100/100 [2:35:34<00:00, 93.34s/it]


In [ ]:
top_n = 10

In [ ]:
top_documents = {}
for query_id, doc_scores in test_dict.items():
  # Directly take the top N documents (already sorted)
  top_documents[query_id] = dict(list(doc_scores.items())[:top_n])

In [ ]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308202 sha256=401768a05709cee3891d2a1c5d839ed20038e8de3db9a36cce3baf8cc5f524eb
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [ ]:
import pytrec_eval

In [ ]:
converted_qrels = {str(k): {str(inner_k): inner_v for inner_k, inner_v in v.items()} for k, v in qrels.items()}


In [ ]:
converted_top_documents = {str(k): {str(inner_k): inner_v for inner_k, inner_v in v.items()} for k, v in top_documents.items()}


In [ ]:
converted_qrels

{'1': {'31715818': 1},
 '3': {'14717500': 1},
 '5': {'13734012': 1},
 '13': {'1606628': 1},
 '36': {'5152028': 1, '11705328': 1},
 '42': {'18174210': 1},
 '48': {'13734012': 1},
 '49': {'5953485': 1},
 '50': {'12580014': 1},
 '51': {'45638119': 1},
 '53': {'45638119': 1},
 '54': {'49556906': 1},
 '56': {'4709641': 1},
 '57': {'4709641': 1},
 '70': {'5956380': 1, '4414547': 1},
 '72': {'6076903': 1},
 '75': {'4387784': 1},
 '94': {'1215116': 1},
 '99': {'18810195': 1},
 '100': {'4381486': 1},
 '113': {'6157837': 1},
 '115': {'33872649': 1},
 '118': {'6372244': 1},
 '124': {'4883040': 1},
 '127': {'21598000': 1},
 '128': {'8290953': 1},
 '129': {'27768226': 1},
 '130': {'27768226': 1},
 '132': {'7975937': 1},
 '133': {'38485364': 1,
  '6969753': 1,
  '17934082': 1,
  '16280642': 1,
  '12640810': 1},
 '137': {'26016929': 1},
 '141': {'6955746': 1, '14437255': 1},
 '142': {'10582939': 1},
 '143': {'10582939': 1},
 '146': {'10582939': 1},
 '148': {'1084345': 1},
 '163': {'18872233': 1},
 '1

In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(converted_qrels, {'P.3,5,10', 'recall.3,5,10', 'ndcg_cut.3,5,10', 'map_cut.3,5,10'})
result = evaluator.evaluate(converted_top_documents)
print(result)
metrics = ['P','ndcg_cut', 'recall', 'map_cut']
cutoffs = [3,5,10]
scores = {f'{metric}_{cutoff}': 0 for metric in metrics for cutoff in cutoffs}
for key in result:
  for metric in metrics:
    for cutoff in cutoffs:
      scores[f'{metric}_{cutoff}'] += result[key][f'{metric}_{cutoff}']
run_length = len(test_dict)
for score in scores:
  scores[score] /= run_length

{'1': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.0, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.0, 'ndcg_cut_10': 0.0, 'map_cut_3': 0.0, 'map_cut_5': 0.0, 'map_cut_10': 0.0}, '3': {'P_3': 0.0, 'P_5': 0.2, 'P_10': 0.1, 'recall_3': 0.0, 'recall_5': 1.0, 'recall_10': 1.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.38685280723454163, 'ndcg_cut_10': 0.38685280723454163, 'map_cut_3': 0.0, 'map_cut_5': 0.2, 'map_cut_10': 0.2}, '5': {'P_3': 0.3333333333333333, 'P_5': 0.2, 'P_10': 0.1, 'recall_3': 1.0, 'recall_5': 1.0, 'recall_10': 1.0, 'ndcg_cut_3': 1.0, 'ndcg_cut_5': 1.0, 'ndcg_cut_10': 1.0, 'map_cut_3': 1.0, 'map_cut_5': 1.0, 'map_cut_10': 1.0}, '13': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.0, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.0, 'ndcg_cut_10': 0.0, 'map_cut_3': 0.0, 'map_cut_5': 0.0, 'map_cut_10': 0.0}, '36': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.1, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.5, 'ndcg_cut_3': 0.0, 

In [ ]:
scores

{'P_3': 0.18000000000000005,
 'P_5': 0.1259999999999999,
 'P_10': 0.07299999999999991,
 'ndcg_cut_3': 0.4511102728446255,
 'ndcg_cut_5': 0.47871386929347204,
 'ndcg_cut_10': 0.5083768359685152,
 'recall_3': 0.51,
 'recall_5': 0.5758333333333333,
 'recall_10': 0.6583333333333334,
 'map_cut_3': 0.42458333333333337,
 'map_cut_5': 0.4410416666666667,
 'map_cut_10': 0.45548611111111115}

In [ ]:
qrel_pseudo['1']

'Please provide a demonstration of the properties of 0-dimensional biomaterials, focusing on their inductive and inducive characteristics.'

In [ ]:
queries['1']

'0-dimensional biomaterials show inductive properties.'

In [ ]:
with open("/content/drive/MyDrive/646Project/sharedfiles/qrel_pseudo_pretrained_gpt_4omini_wordnet_scifact.csv", "w") as file:
    json.dump(qrel_pseudo, file, indent=4)
print("Data saved as generated_queries.json")
# qrel_pseudo.to_csv('/content/drive/MyDrive/646Project/sharedfiles/qrel_pseudo_pretrained_gpt_4omini_wordnet_scifact.csv')

Data saved as generated_queries.json
